# W207.6 Final Project - Predicting Cancer Type from Tumor Mutations
### Tony Di Sera, Vijay Singh, Rajiv Nair, Jeremey Fraenkel


# Overview

In this project, we analyze the tumor mutation dataset from PanCancer Atlas Initiative https://www.cell.com/pb-assets/consortium/pancanceratlas/pancani3/index.html. This is a cancer dataset comprising over 10,000 patients diagnosed with cancer.  Overall, the study collected diverse and detailed molecular information on each patient's tumor, including DNA sequencing.

#### Primary Dataset
The primary dataset we will be using is the somatic mutations file. This file encodes whether or not a gene was found mutated in the biopsied tumor. In addition, we may pull some patient features like gender and age at diagnosis from the clinical patient file.

Number of Instances:  3,600,963 somatic mutations for 10,956 cancer patients
Number of Attributes:  ~100 attributes for mutations, ~700 clinical attributes for patients. We will aggregate the mutation data by gene for each patient, reducing the number of attributes by patient to ~ 500-1000 features.

#### Background
By comparing the DNA from normal tissue cells to those of the cancerous cells, somatic mutations can be identified and characterized. Somatic mutations are non-inherited variations to the DNA of a cell that arise during an individual's lifetime. We will use these DNA mutations to predict cancer type, classified into 33 different tissue/organ types.  

#### Motivation
There is clinical value in being able to predict cancer type based on molecular profiles.  For some patients diagnosed with cancer, the biopsied tumor doesn't match the histologic characteristics of the organ/tissue site.  For example, a patient may have a liver tumor that cannot be characterized as liver cells when reviewed by the pathologist.  In these cases, the cancer may have originated from another site and has metastasized to the liver.  This is where genomic tumor data may provide insights by predicting the 'cell of origin', leading to a better-suited therapy for the patient.


# Initialization

In [ ]:
import pandas as pd
import urllib.request
import numpy as np
import glob
import os
import warnings
from textwrap import wrap
import matplotlib.pyplot as plt
from IPython.display import display
import time
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from sklearn import preprocessing
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import RFE
from sklearn import preprocessing
from sklearn import metrics
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.model_selection import StratifiedKFold
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
#import tensorflow as tf
#import tensorflow.keras as K
#from tensorflow.keras.layers import Dense as Dense
#from tensorflow.keras.utils import to_categorical
#from tensorflow.keras import regularizers
#from tensorflow.keras.layers import Dropout
#from tensorflow.keras.callbacks import Callback


plt.rcParams.update({'figure.max_open_warning': 0})

# Establish the colors for each cancer type
label_colors = []
cm = plt.get_cmap('tab20b')
for i in range(20):
    label_colors.append(cm(i))
cm = plt.get_cmap('tab20c')
for i in range(13):
    label_colors.append(cm(i))

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')


In [ ]:
cd /content/drive/My Drive/berkeley/W207 machine learning/Final Project/w207_6_sum19_g5_final_project

In [ ]:
if tf.test.gpu_device_name() != '/device:GPU:0':
  print('WARNING: GPU device not found.')
else:
  print('SUCCESS: Found GPU: {}'.format(tf.test.gpu_device_name()))

In [ ]:
# create the directory where the downloaded directory is stored
data_dir = "./data"
if not os.path.isdir(data_dir):
    os.makedirs(data_dir)
    
# create the directory where the metrics are stored
metrics_dir = "./metrics"
if not os.path.isdir(metrics_dir):
    os.makedirs(metrics_dir)
    
# create the raw where the source data is stored
raw_dir = "./raw"
if not os.path.isdir(raw_dir):
    os.makedirs(raw_dir)    

In [ ]:
# This loads the data dictionary to will convert
# the tumor_sample_barcode into a cancer_type
# and provide full names for the cancer types
tcga_dict = open("./data/tcga_dictionaries.txt","r")
dict_name_index = 0 #Set dictionary index counter to 0
for line in tcga_dict:
    if line.startswith("#"): #If line starts with #, the next line will be a known dictionary
        dict_name_index += 1
    elif dict_name_index == 4:
        tissue_source_site = eval(line)            
    elif dict_name_index == 5:
        code_to_disease = eval(line)
    elif dict_name_index == 6:
        disease_to_code = eval(line)

# Data Collection
For our analysis of cancer prediction using gene mutation and clinical data from patients, we will gather data from multiple sources. First we obtain the somatic mutation data from the PanCancerAtlas website (https://gdc.cancer.gov/about-data/publications/pancanatlas). We also download the patient clinical data that corresponds to the tumor data. At this time, we are not bringing in clinical features, but as the project progresses, we would like to bring in a few features from this clinical dataset (e.g. age a diagnosis, gender). In our notebook, we store this data locally so that it does not have to be downloaded if the notebook kernel is restarted and run multiple times.

In [ ]:
# to make this notebook's output stable across runs
np.random.seed(42)

### Download the somatic mutations file
This file is in the 'MAF' file format, a bioinformatics tab separated format that can contains one record
for each mutation observed in a patient tumor sample.

In [ ]:
# This downloads a 753 MB somatic mutations gzip file.  
# This will take about 1-5 mins depending on your
# connection speed.
mutations_filename = "./data/somatic_mutations.maf.gz"
if os.path.isfile(mutations_filename):
    print("Skipping download, as file %s is present" %(mutations_filename))
else:
    print('Downloading mutation data. 753 MB (may take a few minutes)...')
    url = 'http://api.gdc.cancer.gov/data/1c8cfe5f-e52d-41ba-94da-f15ea1337efc'  
    urllib.request.urlretrieve(url, mutations_filename)  
print("done.")

### Download the patient clinical data

In [ ]:
# This downloads an 18 MB patient clinical data file
patient_filename = "./data/patient_clinical_data.txt"
if os.path.isfile(patient_filename):
    print("Skipping download, as file %s is present" %(patient_filename))
else:
    print('Downloading clinical data ...')  
    url = 'http://api.gdc.cancer.gov/data/0fc78496-818b-4896-bd83-52db1f533c5c'
    urllib.request.urlretrieve(url, patient_filename)  
print("done.")

## Loading Gene Mutation Data ##

Here we read the gene mutation data. This data file contains many columns, but after careful curation, we have decided to consider the following colums:

1. **tumor_sample_barcode**: this contains the barcode with the first 12 characters identifying the patient
2. **gene**: this is the actual gene that has been mutated (for e.g. TACC2, JAKMIP3, PANX3)
3. **gene_type**: this indicates if the gene is protein coding or not.
4. **chromosome**  **start** **end** **Strand**: the chromosome, start position and end position tells us the location of the gene where the mutation is seen.  Strand indicates if it is on the forward or reverse strand of the DNA.
5. **variant_type**: this indicates if it is a single substitution mutation (SNP), a small deletion (DEL), or small insertion (INS), two nucleotide substitution (DNP), three nucleotide substitution (TNP), or more that three nucleotide substitution (ONP)
6. **variant_classification**: this indicates what kind of molecular effect that this mutation will have on the protein.  The most common classes indicate if the substitution causes a change to the amino acid (missense vs silent).  Nonsense mutations cause premature termination of the protein; frameshift mutations cause a misreading of the amino acid sequence.
7. **variant_impact**: this indicates how damaging the mutation -- HIGH, MODERATE, MODIFIER, or LOW.


In [ ]:
# Load the mutations dataframe
print('Loading mutations dataframe ...')

mutations = pd.read_csv(mutations_filename, compression='gzip',
                        sep='\t',
                        usecols=['Tumor_Sample_Barcode','Hugo_Symbol', 'BIOTYPE',
                                'Chromosome', 'Start_Position',  'End_Position', 'Strand',
                                'Variant_Type',  'Variant_Classification', 'IMPACT' ])

print("done.")

# Set mutations index
mutations['row'] = np.arange(len(mutations))
mutations.set_index('row', inplace=True)

# Rename the columns to more consistent names
renamed_columns = { 'Tumor_Sample_Barcode': 'tumor_sample_barcode', 
                    'Hugo_Symbol': 'gene', 
                    'BIOTYPE': 'gene_type', 
                    'Chromosome': 'chromosome', 
                    'Start_Position': 'start', 
                    'End_Position': 'end', 
                    'Strand': 'strand', 
                    'Variant_Type': 'variant_type', 
                    'Variant_Classification': 'variant_classification', 
                    'IMPACT': 'variant_impact'}
mutations.rename(renamed_columns, inplace=True, axis=1)


print("\nMutations count:       ", mutations.tumor_sample_barcode.count())
print("Number of unique samples:", mutations.tumor_sample_barcode.nunique())


The actual cancer type can be found by parsing the tumor sample barcode and then looking up
the cancer type code in the dictionary based on the tissue source site portion of the
tumor sample barcode. For e.g., the *tumor_sample_barcode* 'TCGA-ZX-AA5X-01A-11D-A42O-09' will be parsed into the *tissue source site* **ZX** that is then mapped using the tissue cancer dictionary to *Cervical_squamous_cell_carcinoma_and_endocervical_adenocarcinoma* or *CESC*.

In [ ]:
# Parse the tissue source site from the tumor sample barcode.  Then use the
# tissue site source to lookup the cancer type from the tcga_dictionaries
def parse_cancer_type(tumor_sample_barcode):
    tss = tumor_sample_barcode.split("-")[1] #Extra the tissue source site from the tcga_id
    cancer_type = disease_to_code[tissue_source_site[tss][1]][0] #Convert from tss to disease to code 
    return cancer_type


mutations['cancer_type'] = mutations['tumor_sample_barcode'].apply(parse_cancer_type)
print("Number of unique cancer types:", mutations.cancer_type.nunique())


# Get the patient barcode.  This is what we will use to join the mutations to the clinical data
def parse_patient_barcode(tumor_sample_barcode):
        return tumor_sample_barcode[0:12]

mutations['patient_barcode'] = mutations['tumor_sample_barcode'].apply(parse_patient_barcode)
#mutations = mutations.drop(['tumor_sample_barcode'], axis=1)
#mutations = mutations.drop(['cancer_type'], axis=1)
print("Number of unique patients:", mutations['patient_barcode'].nunique())

## Loading Patient Data##

Here we load the clinical data. This is data for patients for whom we collected the gene mutation data above. The patients are identified by $patient\_barcode$. We will use this field to populate the gene mutation data from the dataframe above in the table we are about to read. The clinical data has patient information such as gender and age at diagnosis.

In [ ]:
# Load the clinical data
print('Loading clinical dataframe ...')
clinical = pd.read_csv(patient_filename, sep='\t',
                        usecols=['bcr_patient_barcode', 'acronym', 'gender', 
                                 'age_at_initial_pathologic_diagnosis'])

# Rename the columns to more consistent names
renamed_columns = { 'bcr_patient_barcode': 'patient_barcode', 
                    'acronym': 'cancer_type' }
clinical.rename(renamed_columns, inplace=True, axis=1)

print('Clinical count', clinical.patient_barcode.count())

# Get cancer types
cancer_types = clinical['cancer_type'].unique()
print("\nNumber of cancer types", len(cancer_types))

# Get number of cases per cancer type
group_by_patient = clinical.groupby(['cancer_type'])['patient_barcode'].nunique()
print("Number of patients", group_by_patient.sum())
group_by_patient.plot.bar(figsize=(12,4))

In the above histogram, we can already see a challenge that we will be faced with in this work. While some cancers such as BRCA (Breast Cancer) have a large number of cases represented in the dataset, other such as Uterine Carcinosarcoma (UCS) have very few cases. We will pay special attention to the robustness of our classifiers in being able to classify the rarer cases with high accuracy as well.

In [ ]:
# Show cancer types that would be filtered out if examples < 60
display(group_by_patient[group_by_patient <= 60])

## Creating Merged Data ##

Now that we have both gene and cancer data in one dataframe, and the patient clinical data in another dataframe, we will use the **patient_barcode** to merge these into a single table. With this, we can drop the tumor_sample_barcode column, since it has served its purpose. Looking at the data, it seems like some patient data is missing from the gene data. Simultaneously, some data in the gene dataframe does not have corresponding clinical data. Hence our merged dataframe size will be lower than the original mutations dataframe size.

In [ ]:
clinical['patient_barcode'].isnull().values.any()

In [ ]:
missing_count = 0
gene_barcode_set = set(mutations.patient_barcode.unique())
for bc in gene_barcode_set:
    if bc not in set(clinical.patient_barcode.unique()):
        missing_count += 1
print("%d patients with gene data missing in clinical data" %missing_count)

In [ ]:
# perform the merge
merged = mutations.merge(clinical, left_on='patient_barcode', right_on='patient_barcode')
print('Merged mutations count:   ', merged.patient_barcode.count())
print('Number of unique patients:', merged.patient_barcode.nunique())
merged.rename({'cancer_type_x': 'cancer_type'}, axis=1, inplace=True)
print('Number of cancer types:   ', merged.cancer_type.nunique())

In [ ]:
# store the merged data, and the mutations data into csv format
fileName = "./data/mutations_with_clinical.csv"
print("  writing", fileName, "...")
merged.to_csv(fileName)
print("  done.")

fileName = "./data/mutations.csv"
print("  writing", fileName, "...")
mutations.to_csv(fileName)
print("  done.")

## Eliminate any psuedo-genes. 

This is a common filter in bioinformatics analysis, eliminating pseudo-genes.  These are imperfect copies of functional genes.

In [ ]:
# Eliminate psuedo genes
psuedo_genes = list(['transcribed_unprocessed_pseudogene',
               'polymorphic_pseudogene', 
               'unprocessed_pseudogene', 
               'transcribed_processed_pseudogene', 'processed_pseudogene',
               'pseudogene', 'unitary_pseudogene'])

before_count               = mutations.gene.nunique()
mutations_coding           = mutations[~mutations.gene_type.isin(psuedo_genes)]
after_count                = mutations_coding.gene.nunique()
print("Filtered out ", str(before_count - after_count), "genes")
mutations                  = mutations_coding

## Split the data into training and test datasets
Split the data into a training and test split.  We will use a split of 80% training, 20% test.  
We will split based on the patient_barcode.  As part of feature engineering, we will be 
aggregating mutations, so that each example will be represented as a patient (tumor), with
columns for each gene.

In [ ]:
#
# Split the patients into training and test
#
def split_patient_data(data):
    patient_data = data.patient_barcode.unique()

    le     = preprocessing.LabelEncoder()
    patient_labels_string = data.groupby('patient_barcode')['cancer_type'].nunique()
    patient_labels = le.fit_transform(patient_labels_string)
    
    print("Number of unique patients:           ", patient_data.shape[0])
    print("Number of labels for unique patients:", len(patient_labels))
    
    train_data, test_data, train_labels, test_labels = train_test_split(
                                                               patient_data, patient_labels,
                                                               stratify=patient_labels, 
                                                               test_size=0.20)

    print("\ntraining patients:  ", train_data.shape[0])
    print("test patients:      ", test_data.shape[0])
    return {'train_patients': train_data, 'test_patients': test_data}

In [ ]:
#
#  Split Mutations data (based on patient split) and 
#  write out data files
#
def split_and_save_mutation_data(split):   
    train_patients = split['train_patients']
    test_patients  = split['test_patients']

    train_mutations = mutations[mutations.patient_barcode.isin(train_patients)]
    test_mutations  = mutations[mutations.patient_barcode.isin(test_patients)]
    print("\ntraining data:      ", train_mutations.shape[0])
    print("test data:          ", test_mutations.shape[0])
    print("\nall data:           ", test_mutations.shape[0])
    print("train + test:       ", test_mutations.shape[0] + test_mutations.shape[0])
    
    # Write out mutations training data as csv file
    print("\nWriting training set ...")
    train_mutations.to_csv("./data/somatic_mutations_train.csv")
    print("done.")

    # Write out mutations test data as csv file
    print("\nWriting test set ...")
    test_mutations.to_csv("./data/somatic_mutations_test.csv")
    print("done.")

split = split_patient_data(mutations)   
split_and_save_mutation_data(split)

# EDA and Feature Selection



Here, we open the data we put together in the previous notebook. For the initial analysis, we look at $cancer\_type$, $patient\_barcode$, $gene$ and $gene\_type$.

### Load the mutations train and test datasets.

In [ ]:
print('Loading data ...')
mutations = {}
mutations['train'] = pd.read_csv("./data/somatic_mutations_train.csv", 
                             usecols=['cancer_type', 'patient_barcode', 'gene', 'gene_type'])
mutations['test']  = pd.read_csv("./data/somatic_mutations_test.csv", 
                             usecols=['cancer_type', 'patient_barcode', 'gene', 'gene_type'])
print("done.")
print("Mutations training data count:", mutations['train']['patient_barcode'].count())
print("Mutations test data count:    ", mutations['test']['patient_barcode'].count())

### Show distribution of genes across patient tumors

In [ ]:
print("Number of genes across all patient tumors:", mutations['train'].gene.nunique())
print("\n\n")

#
# Show how many genes are mutation in a patient tumor
#
group1         = mutations['train'].groupby(['patient_barcode'])['gene'].nunique().reset_index(name='count')
group1.columns = ['patient', 'gene_count']
ax = group1['gene_count'].hist(bins=200, figsize=(12,4))
_ = ax.set_xlabel("Number of Genes")
_ = ax.set_ylabel("Patient Tumor Frequency")
_ = ax.set_title("Distribution of Genes for a Patient Tumor", fontsize=20)
plt.show()
print('Number of genes per patient tumor')
print("  min, max:", int(group1['gene_count'].min()), ',', int(group1['gene_count'].max()))
print("  mean    :", int(group1['gene_count'].mean()))
print("  median  :", int(group1['gene_count'].median()))
print("\n")

#
# Show now many tumors contain the same gene
#
group2         = mutations['train'].groupby(['gene'])['patient_barcode'].nunique().reset_index(name='count')
group2.columns = ['gene', 'patient_count']
ax = group2['patient_count'].hist(bins=50, figsize=(12,4))
_ = ax.set_xlabel("Number of Patient Tumors")
_ = ax.set_ylabel("Gene Frequency")
_ = ax.set_title("Distribution of Patients for a Gene", fontsize=20)
plt.show()
print('Number of patient tumors with same mutated gene')
print("  min, max:", int(group2['patient_count'].min()), ',', int(group2['patient_count'].max()))
print("  mean    :", int(group2['patient_count'].mean()))
print("  median  :", int(group2['patient_count'].median()))
print("\n")


#
# Show now many cancer types contain the same gene
#
gene_cc_count         = mutations['train'].groupby(['gene'])['cancer_type'].nunique().reset_index(name='count')
gene_cc_count.columns = ['gene', 'cancer_type_count']
gene_cc_count         = gene_cc_count.sort_values(['cancer_type_count', 'gene'], ascending=[0,1])
ax = gene_cc_count['cancer_type_count'].hist(bins=40, figsize=(12,4))
_ = ax.set_xlabel("Number of Cancer Types")
_ = ax.set_ylabel("Gene Frequency")
_ = ax.set_title("Genes in common across many cancer types", fontsize=20)
plt.show()
print('\nNumber of cancer types with same mutated gene')
print("  min, max: ", int(gene_cc_count['cancer_type_count'].min()), ',', int(gene_cc_count['cancer_type_count'].max()))
print("  mean    :", int(gene_cc_count['cancer_type_count'].mean()))
print("  median  :", int(gene_cc_count['cancer_type_count'].median()))

From the histogram above, it is clear that even through we have a large number of genes, only a small number of them are turned on in the patient tumor data that we have. This is the classic problem of a large feature space with a much smaller number of samples. Hence we will need to perform a dimensionality reduction technique such as PCA here.

In [ ]:
# Print out the number of cancer types that are present in the 
# mutations dataset
cancer_types = mutations['train'].cancer_type.unique()
print("\nNumber of cancer types:", len(cancer_types))

# Get number of cases per cancer type
group_patients_by_cancer = mutations['train'].groupby(['cancer_type'])['patient_barcode'].nunique()
print("Number of patients    :", group_patients_by_cancer.sum())
print("\n")
ax = group_patients_by_cancer.plot.bar(figsize=(12,4))
_ = ax.set_title("Number of patient tumors for each cancer type", fontsize=20)

The above chart shows that there are some cancers, such as BRCA and LUAD that have a large representation in our dataset, but other such as DBLC and UCS that are present in much smaller numbers. This will present a challenge for our classifier. Specifically, we want our classifier to be able to classify each of the 32 types of cancers with high precision, but the model should also be able to identify the cancers that don't have a proportionate representation in our data set. It could be that these are cancers are rare, or perhaps they are simply rare in our dataset. **Note:** add more details about the cancers that are abundant as well as rare in this dataset.

In [ ]:
# Get the unique genes per cancer type
group_genes_by_cancer = mutations['train'].groupby(['cancer_type'])['gene'].nunique();

print("Number of genes in each cancer type")
print("  min, max:", int(np.round(group_genes_by_cancer.min())), ',', int(np.round(group_genes_by_cancer.max())))
print("  mean    :", int(np.round(group_genes_by_cancer.mean())))
print("  median  :", int(np.round(group_genes_by_cancer.median())))
print("\n")


ax = group_genes_by_cancer.plot.bar(figsize=(12,4))
_ = ax.set_title("Number of genes for each cancer type", fontsize=20)

The above bar chart gives us an idea of how many genes (features for us) are _on_ for each of the cancer types. Cross referencing this chart with the previous one, we see that for some cancers such as DLBC and UCS we have a fair number of active features, even though the number of cases of such cancers are low. We should be able to person isolated (one-vs-rest) analysis for these cases. However, for other cancers, such as KICH (Kidney Chromophobe) and UVM (Uveal Melanoma) we have both a low occurance rate, and a low number of active features. This second category of cancers will need to be handled with care.

## Create different feature sets

From an initial analysis of our data, we can see that we have a large number of features (binary encoded gene mutations), and the number of features is greater than the number of samples that we have. Even before we try any dimentionality reduction technique such as PCA, we can use other tools to reduce the number of features. One such method is the scikit utility **SelectKBest**. This utility routine can apply the **Chi-Square** test to select the specified number of best features. Another method is to use a LogisticRegression Classifier with L1 regularization and an appropriate C value. This will drive down the coefficients of non-important features to 0, which can then be removed. We try multiple such methods below.

In [ ]:
#
# Create feature matrix each row is a patient tumor; each column is a gene
#
def create_patient_x_gene_matrix(mutations, feature_genes, description, save=True):
    cases = list()
    grouped = mutations.groupby('patient_barcode')
    i = int(0)

    cols = ['case_id', 'cancer_type']
    for gene in feature_genes:
        cols.append(gene)

    for name, group in grouped:
        case = list()
        case.append(name)
        for cc in group.cancer_type.head(1):
            case.append(cc)

        for gene_flag in feature_genes.isin(group.gene.unique()):
            switch = 0
            if gene_flag == True:
                switch = 1
            case.append(switch)
        cases.append(case)

    cases_df = pd.DataFrame(cases)
    cases_df.columns = cols
    print("  ", cases_df.shape)
    
    # Write out transformed data to csv
    if save:
        fileName = "./data/" + description + ".csv"
        print("  writing", fileName, "...")
        cases_df.to_csv(fileName)
        print("  done.")
    
    return cases_df

#
# Create a feature matrix based on most frequent genes in each cancer type
#
def create_feature_matrix(mutations_train, mutations_test, top_n_gene_count, save, description):
    print("Formatting gene matrix with top ", top_n_gene_count, "genes from each cancer type")
    
    # Now try to find the most common genes per cancer type and
    # merge these together to come up with a master list
    cancer_gene_count = mutations_train.groupby(['cancer_type', 'gene'])['patient_barcode'].nunique().reset_index(name='count')
    cancer_gene_count.columns = ['cancer_type', 'gene', 'patient_count']

    # Now create a large matrix, row is the gene, column for each cancer type
    df = pd.DataFrame(cancer_gene_count, columns=['cancer_type', 'gene', 'patient_count'])
    gene_cancer_matrix = pd.pivot_table(df, values='patient_count', index=['gene'],
                         columns=['cancer_type'], aggfunc=np.sum, fill_value=0)

    # Now find the top n genes for each cancer type
    top_genes = []
    idx = 0

    plt.rcParams["figure.figsize"] = (20,20)
    for cancer_type in gene_cancer_matrix.columns:
        sorted_genes = gene_cancer_matrix[cancer_type].sort_values(ascending=False)
        top_rows = sorted_genes[sorted_genes > 0].head(top_n_gene_count)
        for gene, patient_count in top_rows.items():
            top_genes.append(list([cancer_type, gene, patient_count]))

    # Turn this back into a matrix, row is gene, column for each cancer type
    top_df = pd.DataFrame(top_genes, columns=['cancer_type', 'gene', 'patient_count'])
    top_gene_cancer_matrix = pd.pivot_table(top_df, values='patient_count', index=['gene'],
                         columns=['cancer_type'], aggfunc=np.sum, fill_value=0)
    print("  number of genes:", top_gene_cancer_matrix.shape[0])
   
    feature_genes = top_gene_cancer_matrix.index
    create_patient_x_gene_matrix(mutations_train, feature_genes, description + ".train", save)
    create_patient_x_gene_matrix(mutations_test,  feature_genes, description + ".test", save)
    
#
# Create a feature matrix for all genes in tumor mutations
#
def create_all_feature_matrix(mutations_train, mutations_test, save, description):
    print("Formatting gene matrix with for all features")
    #
    # Create feature matrix, each row is patient, columns are genes
    #
    feature_genes = pd.Series(mutations_train.gene.unique())
    feature_matrix_train = create_patient_x_gene_matrix(mutations_train, feature_genes, description + ".train", save)
    feature_matrix_test  = create_patient_x_gene_matrix(mutations_test,  feature_genes, description + ".test", save)
    return feature_matrix_train, feature_matrix_test
  
#
# Run KBestFit to determine most discriminatory genes
#
def get_best_fit_features(feature_matrix, n_features):
    #apply SelectKBest class to extract top n best features
    bestfeatures = SelectKBest(score_func=chi2, k=n_features)
    
    data = feature_matrix.loc[:, (feature_matrix.columns != 'cancer_type') & (feature_matrix.columns != 'case_id')]
    labels_string = feature_matrix['cancer_type']
    
    le = preprocessing.LabelEncoder()
    labels = le.fit_transform(labels_string)
    
    fit = bestfeatures.fit(data,labels)
    dfscores = pd.DataFrame(fit.scores_)
    dfcolumns = pd.DataFrame(data.columns)
    
    #concat two dataframes for better visualization 
    scores_df = pd.concat([dfcolumns,dfscores], axis=1)
    scores_df.columns = ['gene', 'score']
    sorted_scores = scores_df.sort_values(by=['score', 'gene'], ascending=[0,1])
    return sorted_scores.gene.values
 
#
# Create a feature matrix based on genes ranked highest using KBestFit
#
def create_best_fit_feature_matrix(feature_matrix_train, feature_matrix_test,  
                               save, description):
    #
    #  Try BestFit (chi squared test) to find most
    #  important genes
    #
    k_best_fits = [100, 700, 1000, 5000, 8000]
    print("Running KBestFit")
    best_genes_ranked  = get_best_fit_features(feature_matrix_train, 8000)
    
    print("  done.")

    for k_best in k_best_fits:
        print("Creating gene matrix with best fit for", k_best, "features")
        best_genes = best_genes_ranked[:k_best]
        print(len(best_genes))
      
        cancer_type = feature_matrix_train['cancer_type']
        case_id     = feature_matrix_train['case_id']
        data_train  = feature_matrix_train.loc[:, feature_matrix_train.columns.isin(best_genes)]
        final_feature_matrix_train = pd.concat([case_id, cancer_type, data_train], axis=1)

        cancer_type = feature_matrix_test['cancer_type']
        case_id     = feature_matrix_test['case_id']
        data_test   = feature_matrix_test.loc[:, feature_matrix_test.columns.isin(best_genes)]
        final_feature_matrix_test = pd.concat([case_id, cancer_type, data_test], axis=1)

        if save:
            fileName = "./data/" + description  + "_" + str(k_best) + ".train.csv"
            print("  writing", fileName, "...")
            print(" ", final_feature_matrix_train.shape)
            final_feature_matrix_train.to_csv(fileName)
            print("  done.")        

            fileName = "./data/" + description  + "_" + str(k_best) + ".test.csv"
            print("  writing", fileName, "...")
            print(" ", final_feature_matrix_test.shape)
            final_feature_matrix_test.to_csv(fileName)
            print("  done.") 

#
# Create a different feature matrix based on changing L1 regularization strength
#
def create_l1_feature_matrix(train_features, test_features, label_encoder, description, save):
    
    train_first_cols    = train_features[train_features.columns[:2]]
    train_data          = train_features[train_features.columns[3:]]
    train_labels        = label_encoder.fit_transform(train_features.cancer_type)

    test_first_cols    = test_features[test_features.columns[:2]]
    test_data          = test_features[test_features.columns[3:]]
    test_labels        = label_encoder.fit_transform(test_features.cancer_type)

    params = {'C':  [100, 10, 1, .5, .25, .1, .05, .025 ]}
    
    for c_param in reversed(params['C']):
        # Keep this random seed here to make comparison easier.
        np.random.seed(0)

        #
        # Perform Logistic Regression on different C values
        # using L1 regularization
        #
        l1 = LogisticRegression(penalty='l1', tol=.01, 
                            solver="liblinear", multi_class="ovr",
                            max_iter=500, C=c_param)
        # Fit model
        l1.fit(train_data, train_labels) 


        # Get the features with non-zero coefficients.  We will use
        # this list to reduce the features 
        non_zero_sums = np.where(np.sum(l1.coef_, axis=0) != 0)
        names = np.array(list(train_data.columns))
        non_zero_genes = names[non_zero_sums] 


        #
        # Reduce feature size, only keeping features with non-zero weights 
        # found using l1 regularization
        #
        trimmed_train_data = train_data[non_zero_genes]
        trimmed_test_data  = test_data[non_zero_genes]
        
        final_features_train = pd.concat([train_first_cols, trimmed_train_data], axis=1)
        final_features_test =  pd.concat([test_first_cols, trimmed_test_data], axis=1)
        
        if save:
            fileName = "./data/" + description + "_c" + str(c_param) + ".train.csv"
            print("  writing", fileName, "...")
            print(" ", final_features_train.shape)
            final_features_train.to_csv(fileName)
            print("  done.")        

            fileName = "./data/" + description + "_c" + str(c_param) + ".test.csv"
            print("  writing", fileName, "...")
            print(" ", final_features_test.shape)
            final_features_test.to_csv(fileName)
            print("  done.")        
            
#
# Create a feature matrix using recursive feature elimination
#
def create_rfe_feature_matrix(train_features, test_features, label_encoder, 
                              classifier, n_features, n_step,
                              description, save):

    
    train_first_cols    = train_features[train_features.columns[:2]]
    train_data          = train_features[train_features.columns[3:]]
    train_labels        = label_encoder.fit_transform(train_features.cancer_type)

    test_first_cols    = test_features[test_features.columns[:2]]
    test_data          = test_features[test_features.columns[3:]]
    test_labels        = label_encoder.fit_transform(test_features.cancer_type)
    

    rfe = RFE(estimator=classifier, n_features_to_select=n_features, step=n_step, verbose=3)
    rfe.fit(train_data, train_labels)
    
    trimmed_train_data = train_data[train_data.columns[rfe.support_]]
    trimmed_test_data  = test_data[test_data.columns[rfe.support_]]
    
    final_features_train = pd.concat([train_first_cols, trimmed_train_data], axis=1)
    final_features_test =  pd.concat([test_first_cols, trimmed_test_data], axis=1)
    

    if save:
        fileName = "./data/" + description +  ".train.csv"
        print("  writing", fileName, "...")
        print(" ", final_features_train.shape)
        final_features_train.to_csv(fileName)
        print("  done.")        

        fileName = "./data/"+ description + ".test.csv"
        print("  writing", fileName, "...")
        print(" ", final_features_test.shape)
        final_features_test.to_csv(fileName)
        print("  done.")        
    
    return final_features_train, final_features_test

### All genes

Create a feature matrix,  Feature matrix will have one row per patient tumor, 
column for every gene encountered in training data set.

In [ ]:
all_train_file = './data/features_all.train.csv'
all_test_file = './data/features_all.test.csv'
if os.path.isfile(all_train_file) and os.path.isfile(all_test_file):
    print("Skipping generation, files %s and %s are present." %(all_train_file, all_test_file))
else:
    feature_matrix_train, feature_matrix_test = create_all_feature_matrix(mutations['train'], 
                                                    mutations['test'], True, 'features_all')

### Top n genes most frequent in each cancer type

Create a feature matrix, getting the top n genes that are most frequent
per label (cancer type).  Merge these genes and create a feature matrix,
one row per patient tumor, column for each merged gene

In [ ]:
create_feature_matrix(mutations['train'], mutations['test'], 100, True, 'features_top_100_genes')

### KBestFit 

Create a feature matrix, using sklearn BestFit to find top 100, 700, 1000, 5000, 8000 genes. Feature matrix will have one row per patient tumor, column for each 'bestfit' gene



In [ ]:
create_best_fit_feature_matrix(feature_matrix_train, feature_matrix_test, True, 
                          'features_bestfit')


### Logistic Regression (L1)

Trim the features using Logistic Regression, L1 regularization

In [ ]:
label_encoder            = preprocessing.LabelEncoder()
create_l1_feature_matrix(feature_matrix_train, feature_matrix_test, label_encoder,
           'features_l1reg', True)

### RFE

In [ ]:
label_encoder            = preprocessing.LabelEncoder()

svm = LinearSVC(penalty='l2', C=0.1)
lr = LogisticRegression(penalty='l1', C=0.1)

n_features = [100, 800, 4000, 8000]
n_step = .05

label_encoder            = preprocessing.LabelEncoder()

for n_feature in n_features:
    
  _,_ = create_rfe_feature_matrix(feature_matrix_train, 
                           feature_matrix_test,
                           label_encoder,
                           svm, n_feature, n_step,
                           'features_rfe_svm_'  + str(n_feature),
                           True)
  _,_ = create_rfe_feature_matrix(feature_matrix_train, 
                         feature_matrix_test,
                         label_encoder,
                         lr, n_feature, n_step,
                         'features_rfe_lr_'  + str(n_feature),
                         True)


### PCA for dimensionality reduction

In [ ]:
pca = PCA()
all_features_train = feature_matrix_train.drop(columns=['case_id', 'cancer_type'])
pca.fit(all_features_train)
ev = np.cumsum(pca.explained_variance_ratio_)
evcount = len(ev[ev<99.0])
print("Number of features that explain 99% of the variance: ", evcount)
pca = PCA(n_components=evcount)
pca.fit(all_features_train)
train_PCA = pca.transform(all_features_train)
all_features_test = feature_matrix_test.drop(columns=['case_id', 'cancer_type'])
test_PCA = pca.transform(all_features_test)

In [ ]:
train_PCA_df = pd.DataFrame(train_PCA)
train_PCA_df['case_id'] = feature_matrix_train['case_id']
train_PCA_df['cancer_type'] = feature_matrix_train['cancer_type']

test_PCA_df = pd.DataFrame(test_PCA)
test_PCA_df['case_id'] = feature_matrix_test['case_id']
test_PCA_df['cancer_type'] = feature_matrix_test['cancer_type']

#reorder columns and write out
for df, f_name in zip( (train_PCA_df, test_PCA_df), 
                      ('./data/features_after_pca.train.csv', './data/features_after_pca.test.csv') ):
    cols = df.columns.tolist()
    cols = cols[-2:] + cols[:-2]
    df = df[cols]
    df.to_csv(f_name)

# Run the Classifiers

## Load the data

### The data dictionary
All data source files are downloaded above.  This dataset, is a data dictionary
that will allow us to translate cancer type codes to cancer type names.

In [ ]:
# This loads the data dictionary to will convert
# the tumor_sample_barcode into a cancer_type
# and provide full names for the cancer types
tcga_dict = open("./raw/tcga_dictionaries.txt","r")
dict_name_index = 0 #Set dictionary index counter to 0
for line in tcga_dict:
    if line.startswith("#"): #If line starts with #, the next line will be a known dictionary
        dict_name_index += 1
    elif dict_name_index == 4:
        tissue_source_site = eval(line)            
    elif dict_name_index == 5:
        code_to_disease = eval(line)
    elif dict_name_index == 6:
        disease_to_code = eval(line)

In [ ]:
def getDataAndLabels(name, features, label_encoder):
    labels_string = features.cancer_type
   
    labels        = label_encoder.fit_transform(labels_string)

    # Get rid of the cancer type and patient_barcode columns 
    data = features[features.columns[3:]]

    return {'name': name, 'feature_size': data.shape[1],
            'data': data, 'labels': labels , 'label_encoder': label_encoder }

In [ ]:
print('Loading training data ...')

filepath = "./data/features_l1reg*"

# label encoder
label_encoder   = preprocessing.LabelEncoder()

# get all file names for the feature datasets
train_files = glob.glob(filepath + ".train.csv")
all_train_data = {}

# load all of the files
for filename in train_files:
    
    name = filename[16:-10]
    print(" ", name)
    train_features = pd.read_csv(filename)
    all_train_data[name] = getDataAndLabels(name, train_features, label_encoder)

print("done.")


print('Loading test data ...')

test_files = glob.glob(filepath + ".test.csv")
all_test_data = {}
for filename in test_files:
    
    name = filename[16:-9]
    print(" ", name)
    test_features = pd.read_csv(filename)
    all_test_data[name] = getDataAndLabels(name, test_features, label_encoder)

print("done.")

## Functions for tracking metrics

In [ ]:
def get_saved_metrics():
    metrics_filename = "./metrics/metrics.csv"
    if os.path.isfile(metrics_filename):
        metrics_df = pd.read_csv(metrics_filename)
        metrics =  [row for row in metrics_df.T.to_dict().values()]
        return metrics
    else:
        return []
      
def get_saved_metrics_dataframe():
    metrics_filename = "./metrics/metrics.csv"
    if os.path.isfile(metrics_filename):
        metrics_df = pd.read_csv(metrics_filename)
        return metrics_df
    else:
        return None

def save_metrics(name, classifier, metrics, prf_by_label, confusion_mx):
    
    metrics_df = pd.DataFrame(metrics, columns=['name', 'classifier', 'feature_size', 
                                                'accuracy', 'precision', 'recall', 'f1', 
                                                'time'])
    
    # Write out scores as csv files
    metrics_df.to_csv("./metrics/metrics.csv")
    
    # Write out confusion matrix to csv file    
    confusion_mx_df = pd.DataFrame.from_dict(confusion_mx)
    filename = "./metrics/confusion_" + name + "_" + classifier + ".csv"
    confusion_mx_df.to_csv(filename)
    
    # Write out precision, recall, f1 by class to csv file
    prf_by_label_df = pd.DataFrame(prf_by_label)
    filename = "./metrics/prf_by_class_" + name + "_" + classifier + ".csv"
    prf_by_label_df.to_csv(filename)
    
def get_prf_by_label(name, classifier):
    filename = "./metrics/prf_by_class_" + name + "_" + classifier + ".csv"
    if os.path.isfile(filename):
        prf_by_label_df = pd.read_csv(filename)
        return prf_by_label_df[prf_by_label_df.columns[1:]]
    else:
        return None
    
def get_confusion_matrix(name, classifier):
    filename = "./metrics/confusion_" + name + "_" + classifier + ".csv"
    if os.path.isfile(filename):
        confusion_df = pd.read_csv(filename)
        return confusion_df[confusion_df.columns[1:]]
    else:
        return None
      
def calculate_metrics(name, classifier, feature_size, predict, test_labels, 
                      elapsed_time, metrics):
    # Get precision, recall, f1 scores
    prf_scores          = precision_recall_fscore_support(test_labels, predict, 
                                                          average='weighted')
    acc_score           = accuracy_score(test_labels, predict)
    prf_by_label        = precision_recall_fscore_support(test_labels, predict, 
                                                          average=None)
    classification_rpt  = classification_report(test_labels, predict)

    # Get confusion matrix
    conf_mx             = confusion_matrix(test_labels, predict)

    metrics.append({
     'name':               name,
     'classifier':         classifier,
     'feature_size':       feature_size,
     'accuracy':           acc_score,
     'precision':          prf_scores[0],
     'recall':             prf_scores[1],
     'f1':                 prf_scores[2],
     'time':               elapsed_time 
    })
    save_metrics(name, classifier, metrics, prf_by_label, conf_mx)



## Functions for running different classifiers

In [ ]:
#
# Logistic regression
# 
def getBestParamsLogit(train_data, train_labels):
    #
    # Logistic Regression
    #
    lr = LogisticRegression(penalty='l2', multi_class = 'ovr', solver='liblinear', max_iter=150)
    params = {'C': [0.1, 0.25,  0.5]}
    logit = GridSearchCV(lr, params, cv=5,
                         scoring='accuracy', return_train_score=True)

    # Fit  training data
    logit.fit(train_data, train_labels)  
    # Show the best C parameter to use and the expected accuracy
    print(' Best param:', logit.best_params_)
    print(' Accuracy:  ', np.round(logit.best_score_, 4) )
    
    return logit.best_params_

def run_logistic_regression(train_data, train_labels, test_data, test_labels, 
                            name, hyper_params, metrics, forcerun=False):
  
    existing = [record for record in metrics if record['name'] == name and record['classifier'] == 'lr']
    if (not(forcerun) and len(existing) > 0):
      print("\nLogistic Regression (skipping)")
      return
    
    print("\nLogistic Regression", name)

    start = time.process_time()
    if name in hyper_params and 'lr' in hyper_params[name]:
        best_params_logit = hyper_params[name]['lr']
    else:
        print("Running grid search on Logistic Regression...")
        best_params_logit = getBestParamsLogit(train_data, train_labels)

    # Run logistic regression with L2 regularization on reduced
    # feature set
    lr = LogisticRegression(penalty='l2', tol=.01, max_iter=150, 
                          C=best_params_logit['C'], 
                          solver="liblinear", multi_class="ovr")
    lr.fit(train_data, train_labels) 
    predict = lr.predict(test_data)
    elapsed_time = time.process_time() - start


    calculate_metrics(name, 'lr', train_data.shape[1], predict, test_labels, 
                      elapsed_time, metrics)

    print(" done.")
    
    return

In [ ]:
#
# Linear SVM
#

def getBestParamsSVM(train_data, train_labels):
    #
    # SVM
    #
    classifier = LinearSVC(penalty='l2')

    params = {'C': [0.01, 0.1, 0.5]}
    svm = GridSearchCV(classifier, params, cv=4, 
                       scoring='accuracy', return_train_score=True)

    # Fit  training data
    svm.fit(train_data, train_labels)  
    # Show the best C parameter to use and the expected accuracy
    print(' Best param:', svm.best_params_)
    print(' Accuracy:  ', np.round(svm.best_score_, 4) )
    
    return svm.best_params_
  
def run_linear_svm(train_data, train_labels, test_data, test_labels, 
                   name, hyper_params, metrics, forcerun=False):
    
    existing = [record for record in metrics if record['name'] == name and record['classifier'] == 'svm']
    if (not(forcerun) and len(existing) > 0):
      print("\nLinear SVM (skipping)")
      return
    
    print("\nLinear SVM", name)
    start = time.process_time()
    if name in hyper_params and 'svm' in hyper_params[name]:
        best_params_svm = hyper_params[name]['svm']
    else:
        print("Running grid search on Linear SVM...")
        best_params_svm = getBestParamsSVM(train_data, train_labels)

    svm = LinearSVC(penalty='l2', C=best_params_svm['C'])

    svm.fit(train_data, train_labels,) 
    predict = svm.predict(test_data)
    elapsed_time = time.process_time() - start

    calculate_metrics(name, 'svm', train_data.shape[1], predict, test_labels, 
                      elapsed_time, metrics)
    
    print(" done.")
    return

In [ ]:
#
# Decision tree
#
def run_decision_tree(train_data, train_labels, test_data, test_labels, name, hyper_params, metrics):

    existing = [record for record in metrics if record['name'] == name and record['classifier'] == 'dt']
    if (len(existing) > 0):
      print("\nDecision Tree (skipping)")
      return
    
    print("\nDecision Tree", name)
    start = time.process_time()
    dt = DecisionTreeClassifier()
    
    dt.fit(train_data, train_labels,) 
    predict = dt.predict(test_data)
    elapsed_time = time.process_time() - start


    calculate_metrics(name, 'dt', train_data.shape[1], predict, test_labels, 
                      elapsed_time, metrics)
    
    print(" done.")
    return

In [ ]:
#
# Random forest
#
def run_random_forest(train_data, train_labels, test_data, test_labels, name, hyper_params, metrics):
  
    existing = [record for record in metrics if record['name'] == name and record['classifier'] == 'rf']
    if (len(existing) > 0):
      print("\nRandom Forest (skipping)")
      return
    
    print("\nRandom Forest", name)
    start = time.process_time()
    rf = RandomForestClassifier(n_estimators=500)
    
    rf.fit(train_data, train_labels,) 
    predict = rf.predict(test_data)
    elapsed_time = time.process_time() - start

    calculate_metrics(name, 'rf', train_data.shape[1], predict, test_labels, 
                      elapsed_time, metrics)
    
    print(" done.")
    return

In [ ]:

#
# Neural Net
#
def run_neural_net(train_data, train_labels, test_data, test_labels, name, hyper_params, metrics):
    existing = [record for record in metrics if record['name'] == name and record['classifier'] == 'nn']
    if (len(existing) > 0):
      print("\nNeural Net (skipping)")
      return
    
    print("\nNeural Net", name)
    tr_lab = to_categorical(train_labels)
    test_lab = to_categorical(test_labels)
    
    number_of_classes = len(tr_lab[0])
    
    start = time.process_time()
    model = K.Sequential()
    model.add(Dense(2000, input_dim=train_data.shape[1], activation='relu', 
                    kernel_regularizer=regularizers.l1_l2(l2=0.01,l1=0.01)))
    model.add(Dropout(0.2))
    model.add(Dense(1000, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(400, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(100, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(number_of_classes, activation='sigmoid'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics = ["accuracy"])
    
    
    #model.fit(train_data, tr_lab, epochs=200, batch_size=100)
    if name == 'after_pca':
      n_epochs = 10
    else:
      n_epochs = 140
    hist = model.fit(train_data, tr_lab, epochs=n_epochs, batch_size=100,
                     validation_data=(test_data, test_lab))
    
    
    evaluate = model.evaluate(x = test_data, y = test_lab)
    predict = model.predict(test_data)    
    
    elapsed_time = time.process_time() - start

    calculate_metrics(name, 'nn', train_data.shape[1],
                      np.argmax(predict,1), test_labels, elapsed_time, metrics)

    print(" done.")
    
    
    return


## Run the different classifiers 

In [ ]:
hyper_params = {
    'l1reg_c0.025':         {'lr': {'C': 0.25}, 'svm': {'C': 0.01}}, 
    'l1reg_c0.05':          {'lr': {'C': 0.25}, 'svm': {'C': 0.01}}, 
    'l1reg_c0.1':           {'lr': {'C': 0.25}, 'svm': {'C': 0.01}}, 
    'l1reg_c0.25':           {'lr': {'C': 0.25}, 'svm': {'C': 0.01}}, 
    'l1reg_c0.5':           {'lr': {'C': 0.25}, 'svm': {'C': 0.01}},
    'l1reg_c1':             {'lr': {'C': 0.25}, 'svm': {'C': 0.01}},
    'l1reg_c10':            {'lr': {'C': 0.1},  'svm': {'C': 0.01}},
    'l1reg_c100':           {'lr': {'C': 0.25}, 'svm': {'C': 0.01}},
    
    'top_100_genes':        {'lr': {'C': 0.1},  'svm': {'C': 0.01}},

    'bestfit_med':          {'lr': {'C': 0.1 }, 'svm': {'C': 0.01}},
    'bestfit_large':        {'lr': {'C': 0.1 }, 'svm': {'C': 0.01}},
    
    'all':                  {'lr': {'C': 0.25}, 'svm': {'C': 0.01}},
    'after_pca':            {'lr': {'C': 0.5 }, 'svm': {'C': 0.01}},
    
    'rfe_svm_100':          {'lr': {'C': 0.5},  'svm': {'C': 0.1}}, 
    'rfe_lr_100':           {'lr': {'C': 0.5},  'svm': {'C': 0.1}}, 

    'rfe_svm_800':          {'lr': {'C': 0.5},  'svm': {'C': 0.1}}, 
    'rfe_lr_800':           {'lr': {'C': 0.1},  'svm': {'C': 0.01}}, 
    
    'rfe_svm_4000':         {'lr': {'C': 0.5},  'svm': {'C': 0.01}}, 
    'rfe_lr_4000':          {'lr': {'C': 0.1},  'svm': {'C': 0.01}}, 
    
    'rfe_svm_8000':         {'lr': {'C': 0.25}, 'svm': {'C': 0.01}}, 
    'rfe_lr_8000':          {'lr': {'C': 0.1},  'svm': {'C': 0.01}} 
}

metrics = get_saved_metrics()


for name in all_train_data.keys():
    print("************************")
    print(name)
    print("************************")

    train      = all_train_data[name]
    test       = all_test_data[name]
    
    

    run_logistic_regression(train['data'], train['labels'], test['data'], test['labels'], 
                            name, hyper_params, metrics, forcerun=False)
    
    run_linear_svm(train['data'], train['labels'], test['data'], test['labels'], 
                   name, hyper_params, metrics, forcerun=False)

    run_decision_tree(train['data'], train['labels'], test['data'], test['labels'], 
                   name, hyper_params, metrics)
    
    run_random_forest(train['data'], train['labels'], test['data'], test['labels'], 
                   name, hyper_params, metrics)
    
    run_neural_net(train['data'], train['labels'], test['data'], test['labels'], 
                   name, hyper_params, metrics)
    
    

# Visualize Performance across different feature sets, different classifiers

### Load the metrics data

In [ ]:
metrics_df = get_saved_metrics_dataframe()
metrics_df = metrics_df.sort_values(by=['feature_size', 'name', 'classifier'], ascending=[1,1,1])
metrics_df = metrics_df[metrics_df.columns[1:]]

metrics_df = metrics_df.sort_values(by=['classifier', 'feature_size', 'name'], ascending=[1,1,1])

In [ ]:
colors = {'lr': 'olivedrab', 'svm': 'slateblue', 
          'dt': 'mediumseagreen', 'rf': 'goldenrod',
          'xgb': 'coral', 'nn': 'crimson'}

def plot_classifier_metrics(metrics_df, label_encoder, description):
    
    plt.rcParams["figure.figsize"] = (20,14)

    labels = []
    for key, group in metrics_df.groupby(['feature_size', 'name']):
        labels.append(str(key[0]) + '\n' + key[1])
    

    sorted_df_report = metrics_df.sort_values(by=['classifier', 'feature_size', 'name'], ascending=[1,1,1])


        
    for classifier, group in sorted_df_report.groupby(['classifier']):

        plt.plot(labels, group.precision.values, color=colors[classifier], 
                 linewidth=2, label=classifier + " precision", marker='o' )
        plt.plot(labels, group.recall.values, color=colors[classifier], linestyle="dashed",
                 linewidth=2, label=classifier + " recall", marker='o' )
    

    plt.yticks(np.arange(0, .70, .01))
    plt.title(description, fontsize=20)
    plt.ylabel('Precision, Recall', fontsize=14)
    plt.xlabel('Feature Sets', fontsize=14, labelpad=14)
    plt.xticks(rotation='vertical')
    plt.legend()
    plt.grid()
    plt.show()
    
    for classifier, group in sorted_df_report.groupby(['classifier']):

        plt.plot(labels, group.accuracy.values, color=colors[classifier], 
                 linewidth=2, label=classifier + " accuracy", marker='o' )
    

    plt.yticks(np.arange(0, .70, .01))
    plt.ylabel('Accuracy', fontsize=14)
    plt.xticks(rotation='vertical')
    plt.xlabel('Features and Classifiers', fontsize=14, labelpad=20)
    plt.legend()
    plt.grid()
    plt.show()
    
def plot_classifier_times(metrics_df, label_encoder, description):
    
    plt.rcParams["figure.figsize"] = (20,6)

    labels = []
    for key, group in metrics_df.groupby(['feature_size', 'name']):
        labels.append(str(key[0]) + '\n' + key[1])
        
    sorted_df_report = metrics_df.sort_values(by=['classifier', 'feature_size', 'name'], ascending=[1,1,1])


        
    for classifier, group in sorted_df_report.groupby(['classifier']):

        plt.plot(labels, group.time.values, color=colors[classifier], 
                 linewidth=2, label=classifier + " precision", marker='o' )
        
    

    plt.ylabel('Time to train and predict', fontsize=14)
    plt.xlabel('Feature Sets', fontsize=14, labelpad=14)
    plt.xticks(rotation='vertical')
    plt.legend()
    plt.grid()
    plt.show()
    

    
def coords_of_max(theArray, n):
    # Flatten the 2D array
    flat = theArray.flatten()
    # Partition so that the we know the sort order for
    # the cells with the highest values.  We just
    # care about the top n highest values.  So for example,
    # if n = 3, get return 3 indices.  
    indices = np.argpartition(flat, -n)[-n:]
    # Reverse so that we show index of highest value first
    # (descending)
    indices = indices[np.argsort(-flat[indices])]
    # Now return the coordinates for these indices
    # for a 2D array.  This will return 2 arrays,
    # the first for the row index, the second for the
    # column index.  The row index represents the
    # actual digit, the column index represents
    # the confused digit
    return np.unravel_index(indices, theArray.shape)
  


### Plot precision metrics across different classifiers and feature sets

In [ ]:
label_encoder   = preprocessing.LabelEncoder()

metrics_df = metrics_df[~metrics_df.name.isin(['cna_alone'])]
                        
somatic_metrics = metrics_df[~metrics_df.name.isin([
                                                    'cna_l1reg_c0.025', 
                                                    'cna_l1reg_c0.01', 
                                                    'cna_binary_l1reg_c0.025',
                                                    'cna_binary_l1reg_c0.01'])]
cna_metrics     = metrics_df[metrics_df.name.isin( ['cna_l1reg_c0.025', 
                                                    'cna_l1reg_c0.01', 
                                                    'cna_binary_l1reg_c0.025',
                                                    'cna_binary_l1reg_c0.01'])]
# Plot precision, recall, accuracy across different classifiers
# for somatic mutations
plot_classifier_metrics(somatic_metrics, label_encoder, 'Somatic mutations')

# Plot time across different classifiers
# for somatic mutations
plot_classifier_times(somatic_metrics, label_encoder, 'Somatic mutations')


### Report the precision, recall, and f1 score across different classifiers and feature sets

In [ ]:
def show_precision_recall_by_label(prf_by_label_df, name, classifier, label_encoder):
    
    plt.rcParams["figure.figsize"] = (16,14)
    
    labels = []
    for i in range(prf_by_label_df.shape[1]):
        label = label_encoder.inverse_transform([i])[0]
        labels.append(label)
    
    
    y_pos = np.arange(len(labels))    

    fig, (ax1, ax2) = plt.subplots(ncols=2, sharey=False)

    ax1.invert_xaxis()
    ax1.invert_yaxis()
    ax1.yaxis.tick_right()
    
    ax1.set_yticks(y_pos)
    ax1.set_yticklabels(labels)
    
    ax2.invert_yaxis()
    ax2.set_yticks(y_pos)
    ax2.set_yticklabels(labels)
        
    ax1.barh(y_pos, prf_by_label_df.iloc[0].values, color=label_colors , label="precision")
    ax2.barh(y_pos, prf_by_label_df.iloc[1].values, color=label_colors,  label='recall')

    ax1.set_title('Precision( ' + classifier + ')')
    ax2.set_title('Recall (' + classifier + ')')
    ax1.grid()
    ax2.grid()
    

    plt.show()



# best precision
sorted_df = somatic_metrics.sort_values(by='precision', ascending=0)
best_precision = sorted_df.head(1)

# best recall
sorted_df = somatic_metrics.sort_values(by='recall', ascending=0)
best_recall = sorted_df.head(1)

# best f1
sorted_df = somatic_metrics.sort_values(by='f1', ascending=0)
best_f1 = sorted_df.head(1)

# best accuracy
sorted_df = somatic_metrics.sort_values(by='accuracy', ascending=0)
best_accuracy = sorted_df.head(1)


# Show the feature set and classifier with the best 
# precision, recall, and f1 scores
print("\n\nBest precision")
display(best_precision)
print("\n\nBest recall")
display(best_recall)
print("\n\nBest f1")
display(best_f1)
print("\n\nBest accuracy")
display(best_accuracy)

# get the scores by label and confusion matrix
# for the best prediction
best_prediction = best_precision
best_name       = best_prediction.name.values[0]
best_classifier = best_prediction.classifier.values[0]


best_prf_by_label_df = get_prf_by_label(best_name, best_classifier)
best_confusion_mx_df = get_confusion_matrix(best_name, best_classifier)

feature_matrix = pd.read_csv("./data/features_" + best_name + ".train.csv")
label_encoder.fit(feature_matrix.cancer_type.unique())

# show a side-by-side barchart of precision and recall for each label
print("\n\nPrecision and Recall by Label with Best F1 score ")
print("Classifier:", best_classifier, "Feature set:", best_name)
show_precision_recall_by_label(best_prf_by_label_df,
                               best_name, best_classifier, label_encoder)
                                                      


## Visualizations for Diagnosing Poor Performing Classes

### How many genes are in common for a cancer type?

In [ ]:
features       = feature_matrix[feature_matrix.columns[1:]]
cancer_types   = sorted(features.cancer_type.unique())
best_precision = np.round(best_prf_by_label_df.loc[0:0].values[0], 2)  
best_recall    = np.round(best_prf_by_label_df.loc[0:1].values[0], 2)  

plt.rcParams["figure.figsize"] = (20,20)
fig = plt.figure()
suptitle = fig.suptitle("Number of Genes in Common for Samples of a Cancer Type", fontsize=20)
ax = fig.subplots(7, 5, sharex=False, sharey=False, squeeze=True)
plt.subplots_adjust(hspace=0.4)
ax = ax.flatten()
_ = ax[33].axis('off')
_ = ax[34].axis('off')


num_samples_per_cancer_type = []
num_genes_per_cancer_type   = []
num_multisample_genes_per_cancer_type = []
pct_multisample_genes_per_cancer_type = []
avg_num_samples_sharing_genes = []


for idx, cancer_type in enumerate(cancer_types, start=0):
  features_ct = features.loc[features.cancer_type == cancer_type]
  features_ct = features_ct[features_ct.columns[2:]]

  print(".", end='')

  
  gene_sums_all = features_ct.sum(axis=0) 
  gene_sums_all = gene_sums_all[gene_sums_all > 0]
  gene_sums = gene_sums_all[gene_sums_all > 1]
  gene_sums.columns = ['gene_count']

  
  num_samples_per_cancer_type.append(features_ct.shape[0])
  num_genes_per_cancer_type.append(gene_sums_all.shape[0])
  num_multisample_genes_per_cancer_type.append(gene_sums.shape[0])
  pct_multisample_genes_per_cancer_type.append((gene_sums.shape[0] / gene_sums_all.shape[0]) * 100)
  avg_num_samples_sharing_genes.append(gene_sums.mean(axis=0))
  
  bins = np.arange(31) - 0.5
  _ = gene_sums.hist(ax=ax[idx], bins=bins, density=True, range=[0,31], color=label_colors[idx] )
  _ = ax[idx].set_title(cancer_type + "\n" 
                        + str(best_precision[idx]) + " prec, " + str(best_recall[idx]) + " recall \n"
                        + str(gene_sums_all.shape[0]) + " genes, " + str(features_ct.shape[0]) + " samples\n" 
                        + str(int(np.round(gene_sums.shape[0] / gene_sums_all.shape[0], 2) * 100)) + "% genes in mult. samples ")
  
  _ = ax[idx].axvline(gene_sums.mean(), color='k', linestyle='dashed', linewidth=1)
  _ = ax[idx].set_xlabel("Number of samples with genes in common")
  _ = ax[idx].set_ylim((0,1))
  _ = ax[idx].set_xticks(np.arange(0, 30, 3))
  _ = ax[idx].set_ylabel("Number of genes in common")

_ = fig.tight_layout()
_ = suptitle.set_y(1.05)
_ = fig.subplots_adjust(top=.95)
plt.show()


### How many genes are common across all cancer types?

In [ ]:
plt.rcParams["figure.figsize"] = (20,20)
fig = plt.figure()
suptitle = fig.suptitle("Number of Genes for a Cancer Type that are common across other Cancer Types", fontsize=20)
ax = fig.subplots(7, 5, sharex=False, sharey=False, squeeze=True)
_ = plt.subplots_adjust(hspace=0.4)
ax = ax.flatten()
_ = ax[33].axis('off')
_ = ax[34].axis('off')

mean_common_cancer_types = []

features_by_cc = features[features.columns[1:]].groupby(['cancer_type']).sum()
for col in features_by_cc.columns:
  features_by_cc[col] = features_by_cc[col].apply(lambda x: 0 if x == 0 else 1)


diff_pairings = []

for idx, cancer_type in enumerate(cancer_types, start=0):
  print(".", end='')
  features_ct = features_by_cc.loc[[cancer_type]]
  
  gene_counts_ct = features_ct.T
  non_zero_genes = list(gene_counts_ct[gene_counts_ct[cancer_type] > 0].index)
  
  other_cancer_types = [c for c in cancer_types if c != cancer_type]
  features_other = features_by_cc.loc[other_cancer_types, non_zero_genes]
  
  gene_sums_ct = features_ct.sum(axis=0) 
  gene_sums_ct = gene_sums_ct[gene_sums_ct > 0]
  gene_sums_ct.columns = ['gene_count']
  
  gene_sums_other = features_other.sum(axis=0) 
  gene_sums_other = gene_sums_other[gene_sums_ct > 0]
  gene_sums_other.columns = ['gene_count']
  
  sums_other     = features_other.sum(axis=0)
  
  mean_common_cancer_types.append(sums_other.mean())
  
  
  diff_pairing       = []
  diff_pairing_norm  = []
  for x, cancer_type_pairing in enumerate(cancer_types, start=0):
    features_pairing    = features_by_cc.loc[[cancer_type_pairing]]
    gene_counts_pairing = features_pairing.T
    non_zero_target     = set(non_zero_genes)
    non_zero_other      = list(gene_counts_pairing[gene_counts_pairing[cancer_type_pairing] > 0].index)
    non_zero_other      = set(non_zero_other)
  
    diff = non_zero_target - non_zero_other
    
    diff_pairing.append(len(diff))
    
  diff_pairings.append(diff_pairing)
  diff_pairings_norm.append(diff_pairing_norm)
  
  bins = np.arange(32) - 0.5
  _ = sums_other.hist(ax=ax[idx], bins=bins, range=[0,33], color=label_colors[idx])
  _ = ax[idx].set_ylim((0,1500))
  _ = ax[idx].axvline(sums_other.mean(), color='k', linestyle='dashed', linewidth=1)
  _ = ax[idx].set_title(cancer_type + " (" + str(gene_sums_ct.shape[0]) + " genes, "
                       + "prec=" + str(best_precision[idx]) + ")")
  
  _ = ax[idx].set_title(cancer_type + "\n" 
                        + str(best_precision[idx]) + " prec, " + str(best_recall[idx]) + " recall \n"
                        + str(gene_sums_ct.shape[0]) + " genes, " + str(num_samples_per_cancer_type[idx]) + " samples")
  
  _ = ax[idx].set_xlabel("Number of Cancer types")
  _ = ax[idx].set_ylabel("Number of Shared genes")

  
fig.tight_layout()
suptitle.set_y(1)
fig.subplots_adjust(top=.95)
plt.show()




In [ ]:
plt.rcParams["figure.figsize"] = (8,4)
fig = plt.figure()
suptitle = fig.suptitle("Is Cancer Type Precision Affected by Number of Samples or Number of Genes?", 
                        fontsize=16)
ax = fig.subplots(1, 2, sharex=False, sharey=False, squeeze=True)

_ = ax[0].scatter(num_samples_per_cancer_type, best_precision, color=label_colors)
_ = ax[0].set_ylabel("Precision")
_ = ax[0].set_xlabel("Number of Samples")

_ = ax[1].scatter(num_genes_per_cancer_type, best_precision, color=label_colors)
_ = ax[1].set_ylabel("Precision")
_ = ax[1].set_xlabel("Number of Genes")


plt.rcParams["figure.figsize"] = (12,4)
fig = plt.figure()
suptitle = fig.suptitle("Is Cancer Type Precision Affected by How Many Samples Share the same Genes?",
                        fontsize=16)
ax = fig.subplots(1, 3, sharex=False, sharey=False, squeeze=True)

_ = ax[0].scatter(num_multisample_genes_per_cancer_type, best_precision, color=label_colors)
_ = ax[0].set_ylabel("Precision")
_ = ax[0].set_xlabel("Number of Genes of Cancer Type\nthat have Samples in Common")


_ = ax[1].scatter(pct_multisample_genes_per_cancer_type , best_precision, color=label_colors)
_ = ax[1].set_ylabel("Precision")
_ = ax[1].set_xlabel("% of all Genes of Cancer Type\nthat have Samples in Common")


_ = ax[2].scatter(avg_num_samples_sharing_genes, best_precision, color=label_colors)
_ = ax[2].set_ylabel("Precision")
_ = ax[2].set_xlabel("Avg Number of Samples of Cancer Type\nthat have Genes Common ")



plt.rcParams["figure.figsize"] = (5,4)
fig = plt.figure()
suptitle = fig.suptitle("Is Cancer Type Precision Affected by How Common Genes are Across different Cancers?", 
                        fontsize=16)
ax = fig.subplots(1, 1, sharex=False, sharey=False, squeeze=True)

_ = ax.scatter(mean_common_cancer_types, best_precision, color=label_colors)
_ = ax.set_ylabel("Precision")
_ = ax.set_xlabel("Number of Cancer Types sharing genes")




### Pairwise comparison of cancer types, how unique is the set of genes for each cancer type?

In [ ]:

informative_labels = []
for i in range(len(cancer_types)):
  informative_labels.append(cancer_types[i] + " (recall=" + str(best_recall[i]))
  
def plot_pairwise_comparison(pairings, title):
  
  pairing_df = pd.DataFrame(pairings, columns=informative_labels, index=informative_labels)
  pairing_df['recall'] = best_prec

  plt.rcParams["figure.figsize"] = (12,12)
  fig = plt.figure()
  ax = fig.add_subplot(111)
  cax = ax.matshow(pairing_df)
  the_title = plt.title(title, fontsize=20)
  _ = fig.colorbar(cax)
  _ = ax.set_xticks(np.arange(0, 33, 1.0))
  _ = ax.set_yticks(np.arange(0, 33, 1.0))
  _ = ax.set_xticklabels(informative_labels, rotation='vertical')
  _ = ax.set_yticklabels(informative_labels)
  _ = plt.xlabel('Compared to')
  _ = plt.ylabel('Cancer Type')
  _ = the_title.set_y(1.18)
  _ = fig.subplots_adjust(top=.95)
  plt.show()

plot_pairwise_comparison(diff_pairings, "Unique genes")


### Show the confusion matrix for the best performing classifier/feature set

In [ ]:
def show_confusion_matrix(conf_mx, label_encoder):

  
    # Determine the error rates for each misclassification pair
    row_sums = conf_mx.sum(axis=1, keepdims=True)
    norm_conf_mx = conf_mx / row_sums
    # Set the error rates for correctly classified pairs (the diagonal) to zero
    np.fill_diagonal(norm_conf_mx, 0)
    
    
    plt.rcParams["figure.figsize"] = (12,12)
    fig = plt.figure()
    ax = fig.add_subplot(111)
    cax = ax.matshow(norm_conf_mx)
    _ = plt.title('Confusion matrix')
    _ = fig.colorbar(cax)
    _ = ax.set_xticks(np.arange(0, 33, 1.0))
    _ = ax.set_yticks(np.arange(0, 33, 1.0))
    _ = ax.set_xticklabels(cancer_types, rotation='vertical')
    _ = ax.set_yticklabels(cancer_types)
    _ = plt.xlabel('Predicted')
    _ = plt.ylabel('True')
    plt.show()
    
    max_coords = coords_of_max(norm_conf_mx, 50)
    confusion_rows = []
    for i in range(len(max_coords[0])):

        # This is the actual label
        actual_label_idx  = max_coords[0][i]
        actual_label      = label_encoder.inverse_transform([actual_label_idx])[0]

        # This is the predicted label
        predicted_label_idx = max_coords[1][i]
        predicted_label = label_encoder.inverse_transform([predicted_label_idx])[0]
        
        # This is the error rate
        error_rate  = norm_conf_mx[max_coords[0][i], max_coords[1][i]]
        error_count = conf_mx[max_coords[0][i], max_coords[1][i]]

        row = list([ actual_label,                     
                     predicted_label,
                     code_to_disease[actual_label][0], 
                     code_to_disease[predicted_label][0], 
                     error_rate, 
                     error_count ])
        confusion_rows.append(row)
    
    df = pd.DataFrame(confusion_rows, columns=['actual', 'predicted',  'actual_name', 'predicted_name', 'error_rate', 'error_count'])
    display(df)
    

cols = [c for c in best_confusion_mx_df.columns]
best_confusion_mx = best_confusion_mx_df[cols].values
show_confusion_matrix(best_confusion_mx, label_encoder)                                                      

### TSNE Visualization

In [ ]:
from sklearn.manifold import TSNE
import matplotlib as mpl

label_encoder   = preprocessing.LabelEncoder()

labels  = list(feature_matrix.cancer_type.unique())
labels.sort()
X       = feature_matrix[feature_matrix.columns[3:]]
label_encoder.fit(labels)

print("Plot TSNE...")
N = len(labels)
colors = mpl.cm.rainbow(np.linspace(0, 1, N))
fig, axes = plt.subplots(nrows=11, ncols=3, figsize=(11, 33))
_ = plt.tight_layout()

for i, ax in enumerate(axes.flat):
    if (i < len(labels)):
      ax.title.set_text("cancer = {}".format(labels[i]))
      d = X[feature_matrix['cancer_type'] == labels[i]]
      dd = TSNE(n_components=2).fit_transform(d)
      _ = ax.scatter(dd[:,0], dd[:,1], color=label_colors[i])
plt.show()